In [1]:
%run common.ipynb

In [2]:
from pyspark.sql import SparkSession


# Initialize Spark Session with MySQL JDBC Driver
sparkCustomer = SparkSession.builder.appName("viewCustomer")\
        .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33")\
        .getOrCreate()


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ffee2f79-c7fa-4f6f-b660-96da77ace3e3;1.0
	confs: [default]
mysql#mysql-connector-java;8.0.33 is relocated to com.mysql#mysql-connector-j;8.0.33. Please update your dependencies.
	found mysql#mysql-connector-java;8.0.33 in central
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
downloading https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/8.0.33/mysql-connector-j-8.0.33.jar ...
	[SUCCESSFUL ] com.mysql#mysql-connector-j;8.0.33!mysql-connector-j.jar (3254ms)
downloading https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/3.21.9/protobuf-java-3.21.9.jar ...
	[SUCCESSFUL ] com.google.protobuf#protobuf-java;3.21.9!protobuf-java.jar(bundle) (1857ms)
:: resolution report :: resol

In [3]:
print(sparkCustomer)

In [4]:
print(USER_MYSQL_URL)
print(MYSQL_PROPERTIES)

jdbc:mysql://172.17.0.2:3306/UserService
{'user': 'root', 'password': '*dipak9', 'driver': 'com.mysql.cj.jdbc.Driver', 'allowPublicKeyRetrieval': 'true', 'useSSL': 'false'}


In [5]:
# Read MySQL table into PySpark DataFrame
customer_df = sparkCustomer.read.jdbc(url=USER_MYSQL_URL, table="Customers", properties=MYSQL_PROPERTIES)
order_df = sparkCustomer.read.jdbc(url=ORDER_MYSQL_URL, table="Orders", properties=MYSQL_PROPERTIES)
products_df = sparkCustomer.read.jdbc(url=PRODUCT_MYSQL_URL, table="Products", properties=MYSQL_PROPERTIES)

# Register as Temp Tables
customer_df.createOrReplaceTempView("customers")
order_df.createOrReplaceTempView("orders")
products_df.createOrReplaceTempView("products")


In [6]:
orderItems_df = sparkCustomer.read.jdbc(url=ORDER_MYSQL_URL, table="Order_Items", properties=MYSQL_PROPERTIES)
orderItems_df.createOrReplaceTempView("Order_Items")

In [7]:
customer_df.show(2)

+-----------+----------+---------+--------------------+--------------------+--------------------+------------+--------+--------+-----------+-------------------+
|customer_id|first_name|last_name|               email|               phone|             address|        city|   state|zip_code|    country|         created_at|
+-----------+----------+---------+--------------------+--------------------+--------------------+------------+--------+--------+-----------+-------------------+
|          1|   Michael|  Rodgers|katiecasey@exampl...|+1-664-301-3316x9...|USS Dudley\nFPO A...| North Traci|   Idaho|   38917|Saint Lucia|2025-03-15 07:34:22|
|          2|     Paula|    Berry|carolyn15@example...|  971-396-8026x98919|PSC 9301, Box 951...|Hornechester|Michigan|   14169|     Turkey|2025-03-15 07:34:32|
+-----------+----------+---------+--------------------+--------------------+--------------------+------------+--------+--------+-----------+-------------------+
only showing top 2 rows



In [8]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- created_at: timestamp (nullable = true)



In [9]:
order_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- total_amount: decimal(10,2) (nullable = true)



In [10]:
products_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: decimal(10,2) (nullable = true)
 |-- stock_quantity: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)



In [11]:
order_df.head(4)

[Row(order_id=1, customer_id=68, order_date=datetime.datetime(2022, 7, 22, 0, 0), status='Shipped', total_amount=Decimal('236.06')),
 Row(order_id=2, customer_id=458, order_date=datetime.datetime(2021, 9, 13, 0, 0), status='Cancelled', total_amount=Decimal('191.20')),
 Row(order_id=3, customer_id=158, order_date=datetime.datetime(2021, 3, 12, 0, 0), status='Shipped', total_amount=Decimal('227.43')),
 Row(order_id=4, customer_id=199, order_date=datetime.datetime(2020, 10, 18, 0, 0), status='Cancelled', total_amount=Decimal('357.54'))]

In [12]:
order_df.filter((order_df.customer_id == 199) |  (order_df.customer_id == 458)).filter(order_df.status == "Delivered").head(4)

[Row(order_id=435, customer_id=199, order_date=datetime.datetime(2021, 4, 28, 0, 0), status='Delivered', total_amount=Decimal('387.55')),
 Row(order_id=681, customer_id=458, order_date=datetime.datetime(2021, 2, 17, 0, 0), status='Delivered', total_amount=Decimal('168.91')),
 Row(order_id=1627, customer_id=199, order_date=datetime.datetime(2025, 1, 19, 0, 0), status='Delivered', total_amount=Decimal('57.34')),
 Row(order_id=1755, customer_id=458, order_date=datetime.datetime(2023, 12, 13, 0, 0), status='Delivered', total_amount=Decimal('313.09'))]

In [ ]:
# Calculate the top 5 products with the highest sales in each region.

In [13]:
order_complete_df = order_df.filter(order_df.status == "Delivered")

In [14]:
order_complete_df.show()

+--------+-----------+-------------------+---------+------------+
|order_id|customer_id|         order_date|   status|total_amount|
+--------+-----------+-------------------+---------+------------+
|       7|        379|2021-03-18 00:00:00|Delivered|      247.31|
|       9|        648|2020-05-09 00:00:00|Delivered|      319.87|
|      14|        938|2020-06-23 00:00:00|Delivered|      492.61|
|      19|        126|2021-03-09 00:00:00|Delivered|       41.17|
|      29|        743|2025-01-03 00:00:00|Delivered|      179.07|
|      32|        672|2024-02-26 00:00:00|Delivered|      129.59|
|      49|        209|2024-08-16 00:00:00|Delivered|      322.80|
|      55|        469|2021-10-18 00:00:00|Delivered|      418.63|
|      61|        730|2023-05-24 00:00:00|Delivered|      274.24|
|      67|        579|2023-06-11 00:00:00|Delivered|      265.36|
|      68|        873|2022-08-23 00:00:00|Delivered|      467.04|
|      72|        179|2020-04-09 00:00:00|Delivered|      238.93|
|      73|

In [15]:
order_complete_df.createOrReplaceTempView("order_complete")

In [21]:
sparkCustomer.sql("""
            with customerData as (
           select customer_id,sum(total_amount) as spending,count(order_id) as total_order,max(order_date) last_order_date
           from order_complete group by customer_id 
   ),
   rankSpending as(
        select customer_id,spending,total_order,last_order_date, rank() over(order by spending desc) as spends_rank
        from customerData
   )
   select customer_id,spending,total_order,last_order_date,spends_rank from rankSpending where spends_rank < 100
""").show()

25/03/18 19:33:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:33:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:33:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:33:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:33:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:33:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 1

+-----------+--------+-----------+-------------------+-----------+
|customer_id|spending|total_order|    last_order_date|spends_rank|
+-----------+--------+-----------+-------------------+-----------+
|        666|11469.72|         40|2025-01-06 00:00:00|          1|
|        190|11465.77|         40|2024-10-11 00:00:00|          2|
|        335|11213.27|         38|2025-03-14 00:00:00|          3|
|        457|11071.19|         38|2025-01-28 00:00:00|          4|
|        863|11053.31|         35|2024-12-31 00:00:00|          5|
|        577|11013.82|         36|2024-10-31 00:00:00|          6|
|        228|11006.96|         39|2025-02-18 00:00:00|          7|
|        847|10565.05|         36|2024-09-26 00:00:00|          8|
|        717|10425.51|         41|2024-12-12 00:00:00|          9|
|        839|10312.79|         34|2025-02-08 00:00:00|         10|
|        309|10232.12|         46|2025-03-06 00:00:00|         11|
|        846|10159.42|         40|2025-02-07 00:00:00|        

In [23]:
from pyspark.sql.functions import sum,count,max

In [24]:
order_group_sum = order_complete_df.groupBy("customer_id").agg(sum("total_amount").alias("spending"),count("order_id").alias("total_order"),max("order_date").alias("last_order_date"))

In [25]:
order_group_sum.show()

+-----------+--------+-----------+-------------------+
|customer_id|spending|total_order|    last_order_date|
+-----------+--------+-----------+-------------------+
|        148| 4859.68|         21|2024-11-26 00:00:00|
|        496| 5819.39|         27|2025-01-08 00:00:00|
|        833| 5233.27|         18|2024-09-20 00:00:00|
|        540| 4728.19|         19|2025-03-06 00:00:00|
|        243| 3617.70|         17|2025-02-06 00:00:00|
|        879| 6286.23|         27|2025-03-04 00:00:00|
|         65| 8632.09|         38|2025-02-21 00:00:00|
|        481| 7987.46|         30|2025-01-17 00:00:00|
|        804| 3621.17|         18|2024-10-17 00:00:00|
|        133| 6830.57|         23|2024-05-30 00:00:00|
|        633| 4386.72|         20|2024-12-12 00:00:00|
|        876| 9398.92|         34|2025-02-11 00:00:00|
|        597| 7302.42|         25|2025-01-28 00:00:00|
|        744| 5042.58|         18|2025-03-07 00:00:00|
|        126| 7267.06|         28|2024-10-20 00:00:00|
|        7

In [26]:
from pyspark.sql.window import Window

In [32]:
from pyspark.sql.functions import col,rank

In [30]:
windowSpec = Window.orderBy(col("spending").desc())

In [35]:
order_group_sum_rank = order_group_sum.withColumn("spends_rank",rank().over(windowSpec))

In [36]:
order_group_sum_rank.show()

25/03/18 19:47:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:47:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:47:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:47:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:47:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 19:47:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/18 1

+-----------+--------+-----------+-------------------+-----------+
|customer_id|spending|total_order|    last_order_date|spends_rank|
+-----------+--------+-----------+-------------------+-----------+
|        666|11469.72|         40|2025-01-06 00:00:00|          1|
|        190|11465.77|         40|2024-10-11 00:00:00|          2|
|        335|11213.27|         38|2025-03-14 00:00:00|          3|
|        457|11071.19|         38|2025-01-28 00:00:00|          4|
|        863|11053.31|         35|2024-12-31 00:00:00|          5|
|        577|11013.82|         36|2024-10-31 00:00:00|          6|
|        228|11006.96|         39|2025-02-18 00:00:00|          7|
|        847|10565.05|         36|2024-09-26 00:00:00|          8|
|        717|10425.51|         41|2024-12-12 00:00:00|          9|
|        839|10312.79|         34|2025-02-08 00:00:00|         10|
|        309|10232.12|         46|2025-03-06 00:00:00|         11|
|        846|10159.42|         40|2025-02-07 00:00:00|        

In [8]:
df = sparkCustomer.createDataFrame(
    [(14, "Tom"), (23, "Alice"), (16, "Bob")], ["age", "name"])

In [9]:
df.show()

+---+-----+
|age| name|
+---+-----+
| 14|  Tom|
| 23|Alice|
| 16|  Bob|
+---+-----+



In [14]:
sparkdf = SparkSession.builder.appName("ddf").getOrCreate()

25/03/15 08:18:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [15]:
print(sparkdf)

In [19]:
dfOne = sparkdf.createDataFrame(
    [(1,2,(3,14)),(4,5,(6,15)),(7,8,(9,16)),(10,11,(12,13))],["a","b","c"])

In [20]:
dfOne.show()

+---+---+--------+
|  a|  b|       c|
+---+---+--------+
|  1|  2| {3, 14}|
|  4|  5| {6, 15}|
|  7|  8| {9, 16}|
| 10| 11|{12, 13}|
+---+---+--------+



In [21]:
dfOne.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)
 |-- c: struct (nullable = true)
 |    |-- _1: long (nullable = true)
 |    |-- _2: long (nullable = true)



In [26]:
#print columns
customer_df.columns

['customer_id',
 'first_name',
 'last_name',
 'email',
 'phone',
 'address',
 'city',
 'state',
 'zip_code',
 'country',
 'created_at']

In [24]:
#print schema
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- created_at: timestamp (nullable = true)



In [28]:
customer_df.head(5)

[Row(customer_id=1, first_name='Michael', last_name='Rodgers', email='katiecasey@example.net', phone='+1-664-301-3316x91300', address='USS Dudley\nFPO AA 46717', city='North Traci', state='Idaho', zip_code='38917', country='Saint Lucia', created_at=datetime.datetime(2025, 3, 15, 7, 34, 22)),
 Row(customer_id=2, first_name='Paula', last_name='Berry', email='carolyn15@example.com', phone='971-396-8026x98919', address='PSC 9301, Box 9514\nAPO AE 66260', city='Hornechester', state='Michigan', zip_code='14169', country='Turkey', created_at=datetime.datetime(2025, 3, 15, 7, 34, 32)),
 Row(customer_id=3, first_name='Angela', last_name='Lawrence', email='kchung@example.org', phone='616-578-2644x4229', address='884 Brandi Pike Apt. 946\nJonesborough, MI 69095', city='New Kelsey', state='Maryland', zip_code='38467', country='Lesotho', created_at=datetime.datetime(2025, 3, 15, 7, 34, 24)),
 Row(customer_id=4, first_name='Nancy', last_name='May', email='carol17@example.org', phone='823-433-0320x81

In [31]:
customer_df.show(2)

+-----------+----------+---------+--------------------+--------------------+--------------------+------------+--------+--------+-----------+-------------------+
|customer_id|first_name|last_name|               email|               phone|             address|        city|   state|zip_code|    country|         created_at|
+-----------+----------+---------+--------------------+--------------------+--------------------+------------+--------+--------+-----------+-------------------+
|          1|   Michael|  Rodgers|katiecasey@exampl...|+1-664-301-3316x9...|USS Dudley\nFPO A...| North Traci|   Idaho|   38917|Saint Lucia|2025-03-15 07:34:22|
|          2|     Paula|    Berry|carolyn15@example...|  971-396-8026x98919|PSC 9301, Box 951...|Hornechester|Michigan|   14169|     Turkey|2025-03-15 07:34:32|
+-----------+----------+---------+--------------------+--------------------+--------------------+------------+--------+--------+-----------+-------------------+
only showing top 2 rows



In [38]:
customer_df.take(2)

[Row(customer_id=1, first_name='Michael', last_name='Rodgers', email='katiecasey@example.net', phone='+1-664-301-3316x91300', address='USS Dudley\nFPO AA 46717', city='North Traci', state='Idaho', zip_code='38917', country='Saint Lucia', created_at=datetime.datetime(2025, 3, 15, 7, 34, 22)),
 Row(customer_id=2, first_name='Paula', last_name='Berry', email='carolyn15@example.com', phone='971-396-8026x98919', address='PSC 9301, Box 9514\nAPO AE 66260', city='Hornechester', state='Michigan', zip_code='14169', country='Turkey', created_at=datetime.datetime(2025, 3, 15, 7, 34, 32))]

In [44]:
customer_df.limit(1)

DataFrame[customer_id: int, first_name: string, last_name: string, email: string, phone: string, address: string, city: string, state: string, zip_code: string, country: string, created_at: timestamp]

In [45]:
customer_df.head()

Row(customer_id=1, first_name='Michael', last_name='Rodgers', email='katiecasey@example.net', phone='+1-664-301-3316x91300', address='USS Dudley\nFPO AA 46717', city='North Traci', state='Idaho', zip_code='38917', country='Saint Lucia', created_at=datetime.datetime(2025, 3, 15, 7, 34, 22))

In [51]:
customer_df.filter(customer_df.country=="Saint Lucia").show()

+-----------+----------+---------+--------------------+--------------------+--------------------+----------------+--------+--------+-----------+-------------------+
|customer_id|first_name|last_name|               email|               phone|             address|            city|   state|zip_code|    country|         created_at|
+-----------+----------+---------+--------------------+--------------------+--------------------+----------------+--------+--------+-----------+-------------------+
|          1|   Michael|  Rodgers|katiecasey@exampl...|+1-664-301-3316x9...|USS Dudley\nFPO A...|     North Traci|   Idaho|   38917|Saint Lucia|2025-03-15 07:34:22|
|        110|   Gregory|    Walls|josedavis@example...|   231-313-7890x9429|98429 Katherine P...|     Austinburgh|Maryland|   36829|Saint Lucia|2025-03-15 07:34:35|
|        390|     David| Peterson|samuel00@example.net|+1-433-855-2564x0...|57056 Stacy Islan...|East Michaeltown|Virginia|   47324|Saint Lucia|2025-03-15 07:34:35|
+---------

In [ ]:
customer_df.filter(customer_id == )